<a href="https://colab.research.google.com/github/TareDevarsh/machine_learning/blob/master/Stock_market_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from datetime import datetime
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')


In [ ]:
with open('mouse_log.txt') as f:
    lines = f.readlines()

In [ ]:
print(len(lines))

28206


In [ ]:
coordinatesTuple = []
timeEpoch = []


for i in range(len(lines)):
  input_string = lines[i]

  # Define a regular expression pattern
  pattern = r'(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},\d{3}):.*\((\d+,\s*\d+)\)'

  # Use re.search to find the matching groups in the string
  match = re.search(pattern, input_string)

  if match:
      # Extract the matched groups
      date_and_time = match.group(1)
      coordinates = match.group(2)

      timeEpoch.append(datetime.strptime(date_and_time, "%Y-%m-%d %H:%M:%S,%f").timestamp())
      coordinatesTuple.append(list(map(int, re.findall(r'\d+', coordinates))))
  else:
      continue



In [ ]:
#Plot/Create the data for the graph
train = df_close[:data_len]
valid = df_close[data_len:]
valid['Predictions'] = predictions
#Visualize the data
plt.figure(figsize=(20,10))
plt.title('Apple Inc Market Prediction')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Trained', 'Actual Value', 'Predicted Value'], loc='lower right')
plt.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(coordinatesTuple)

scaled_data

array([[0.14064411, 0.36622655],
       [0.14349776, 0.36692147],
       [0.14961272, 0.37039611],
       ...,
       [0.13982878, 0.34746352],
       [0.13982878, 0.34676859],
       [0.13982878, 0.34607366]])

In [ ]:
train_size = int(len(scaled_data) * 0.9)
test_size = len(scaled_data) - train_size
train, test = scaled_data[0:train_size], scaled_data[train_size:len(scaled_data)]

In [ ]:
print(len(train),len(test))

25385 2821


In [ ]:
x_train = []
y_train = []

for i in range(60, len(train)):
    x_train.append(train[i-60:i])
    y_train.append(train[i])

# Convert x_train and y_train to numpy arrays if needed
x_train = np.array(x_train)
y_train = np.array(y_train)


In [ ]:
x_train.shape[0], x_train.shape[1]

(25325, 60)

In [ ]:
input_shape = (x_train.shape[1], x_train.shape[2])

x_train.shape


(25325, 60, 2)

In [ ]:
model = Sequential()
model.add(LSTM(100, return_sequences=True, input_shape=input_shape))
model.add(LSTM(90, return_sequences=True))
model.add(LSTM(60, return_sequences=True))  # Additional LSTM layer
model.add(LSTM(40, return_sequences=True))  # Additional LSTM layer
model.add(LSTM(25, return_sequences=False))
model.add(Dense(25, activation='relu'))
model.add(Dense(2))

In [ ]:
model.compile(optimizer= 'adam', loss='mean_squared_error')

In [14]:
model.fit(x_train, y_train, batch_size=1, epochs =5)

Epoch 1/5
25325/25325 [==============================] - 3219s 127ms/step - loss: 0.0023
Epoch 2/5
25325/25325 [==============================] - 3081s 122ms/step - loss: 8.4258e-04
Epoch 3/5
25325/25325 [==============================] - 3143s 124ms/step - loss: 6.7666e-04
Epoch 4/5
25325/25325 [==============================] - 3127s 123ms/step - loss: 6.5491e-04
Epoch 5/5
25325/25325 [==============================] - 3083s 122ms/step - loss: 6.3116e-04


In [ ]:
test


array([[0.27668539, 0.0261115 ],
       [0.2752809 , 0.02399435],
       [0.27434457, 0.02187721],
       ...,
       [0.40121723, 0.34791814],
       [0.40168539, 0.34791814],
       [0.40215356, 0.34791814]])

In [15]:
x_test = []
y_test = test[60:]  # Assuming 'dataset' is your original data, and 'data_len' is the length of your training data

for i in range(60, len(test)):
    x_test.append(test[i-60:i, :])  # Assuming data_test has shape (690, 2)

x_test = np.array(x_test)

In [22]:
y_test.shape

(2761, 2)

In [19]:
# Assuming you have predictions and y_test
predictions = model.predict(x_test)

# Print shapes for debugging
print("predictions shape:", predictions.shape)
print("y_test shape:", y_test.shape)

# Inverse transform the scaled predictions
predictions_original = scaler.inverse_transform(predictions)

# Assuming your original data was 2D, reshape y_test to match predictions_original
y_test_original = scaler.inverse_transform(y_test[:predictions.shape[0]].reshape(-1, 2))

# Calculate RMSE
rmse = np.sqrt(np.mean((predictions_original - y_test_original)**2))
rmse

87/87 [==============================] - 5s 55ms/step
predictions shape: (2761, 2)
y_test shape: (2761, 2)


55.5267752693298

In [37]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

87/87 [==============================] - 7s 78ms/step


989.6138599932962

In [ ]:
#Plot/Create the data for the graph
train = df_close[:data_len]
valid = df_close[data_len:]
valid['Predictions'] = predictions
#Visualize the data
plt.figure(figsize=(20,10))
plt.title('Apple Inc Market Prediction')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Trained', 'Actual Value', 'Predicted Value'], loc='lower right')
plt.show()

NameError: ignored

In [57]:
new_test = test[61]
new_test = scaler.inverse_transform(new_test.reshape(-1, 2))
new_test

array([[1496., 1217.]])

In [52]:
new_test_data = test[:60]
new_test_data = np.expand_dims(new_test_data, axis=0)
new_test_data.shape

(1, 60, 2)

In [58]:
predictions = model.predict(new_test_data)
predictions = scaler.inverse_transform(predictions)
predictions

1/1 [==============================] - 0s 81ms/step


array([[1493.1392, 1209.0747]], dtype=float32)

In [25]:


#Create a new dataframe
new_test = test[61]
#Get teh last 60 day closing price
last_60_days = new_df[-60:].values
#Scale the data to be values between 0 and 1
last_60_days_scaled = scaler.transform(last_60_days)
#Create an empty list
X_test = []
#Append teh past 60 days
X_test.append(last_60_days_scaled)
#Convert the X_test data set to a numpy array
X_test = np.array(X_test)
#Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#Get the predicted scaled price
pred_price = model.predict(X_test)
#undo the scaling
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

NameError: ignored

In [ ]:
apple_quote2 = rd.DataReader('AAPL', data_source='yahoo',start = '2020-05-23',end='2020-05-23')
apple_quote2['Close']

Date
2020-05-22    318.890015
Name: Close, dtype: float64

In [24]:
!mkdir -p saved_model
model.save('mouseNatural.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
